In [ ]:
import os
import glob
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 0. Compute project root from cwd
cwd = os.getcwd()
project_root = os.path.abspath(os.path.join(cwd, os.pardir))

# 1. Load head‐series excels (skipping header row, parsing day‐first dates)
beemster_dir = os.path.join(project_root, "input_files", "input_beemster")
xlsx_paths = glob.glob(os.path.join(beemster_dir, "*.xlsx"))
head_series = []
for path in xlsx_paths:
    df = pd.read_excel(path,
                       header=None,
                       skiprows=1,
                       usecols=[0,1],
                       names=["timestamp","head"],
                       engine="openpyxl")
    df["timestamp"] = pd.to_datetime(df["timestamp"], dayfirst=True, errors="raise")
    df.set_index("timestamp", inplace=True)
    df = df[~df.index.duplicated(keep="first")]
    name = os.path.splitext(os.path.basename(path))[0]
    head_series.append(df["head"].rename(name))

# Concatenate all head series into a single DataFrame
heads_df = pd.concat(head_series, axis=1)

# Convert head series to meters (assuming the original values are in cm)
heads_df = heads_df / 100

# Importing precipitation and evaporation and setting recharge

In [ ]:
# 2. Load precipitation CSV (full record)
prec_path = os.path.join(project_root, "input_files", "input_prec", "prec_station_249.csv")
prec_df = pd.read_csv(prec_path,
                      header=0,
                      parse_dates=["DATE"],
                      dayfirst=False,
                      index_col="DATE",
                      usecols=["DATE","Precipitation"])
prec_df = prec_df[~prec_df.index.duplicated(keep="first")]
prec_series = prec_df["Precipitation"].rename("prec_station_249")

# 2. Load Evaporation CSV (full record)
evap_path = os.path.join(project_root, "input_files", "input_evap", "evap_station_249.csv")
evap_df = pd.read_csv(evap_path,
                      header=0,
                      parse_dates=["DATE"],
                      dayfirst=False,
                      index_col="DATE",
                      usecols=["DATE","ET"])
evap_df = evap_df[~evap_df.index.duplicated(keep="first")]
evap_series = evap_df["ET"].rename("Evaporation")

# Building the plotly graph
- We build a plotly graph with two Y axis, also keep in mind that the head series are hourly values and the precipitation dataset is the daily cumulative value.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 4. Plot with dual y-axis, precipitation as bars, evaporation as a line
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add head series (hourly) as lines on the left axis
for col in heads_df.columns:
    fig.add_trace(
        go.Scatter(
            x=heads_df.index,
            y=heads_df[col],
            mode="lines",
            name=col
        ),
        secondary_y=False
    )

# Add precipitation (daily) as blue bars on the right axis
fig.add_trace(
    go.Bar(
        x=prec_series.index,
        y=prec_series.values,
        name="Precipitation (daily)",
        marker_color="blue",
        opacity=0.5
    ),
    secondary_y=True
)

# Add evaporation (daily) as an orange line on the right axis
fig.add_trace(
    go.Scatter(
        x=evap_series.index,
        y=evap_series.values,
        mode="lines",
        name="Evaporation (daily)",
        line=dict(color="orange", width=2)
    ),
    secondary_y=True
)

# Label axes
fig.update_yaxes(title_text="Head (m)", secondary_y=False)
fig.update_yaxes(title_text="Precipitation & Evaporation (mm)", secondary_y=True)

fig.update_layout(
    title="Hourly Head Series with Daily Precipitation (bars) and Evaporation (line)",
    xaxis_title="Date",
    legend=dict(orientation="h", y=1.02, x=1),
    bargap=0.1
)

fig.show()

# Resampling the head series

In [ ]:
head_resampled = heads_df.resample("D").mean()

In [ ]:
head_resampled.head()

In [ ]:
head_resampled.info()

In [ ]:
prec_series.plot()

### Code om te loopen over de test batch
Bouwt alle modellen die in Pastas mogelijk zijn, en laat daarvan de test scores zien.

In [ ]:
import pastas as ps
import pandas as pd

# Pre‐calculate net input (for “Direct” case)
recharge = prec_series - evap_series

# Model specifications
recharge_models = {
    "Linear":      ps.rch.Linear(),
    "FlexModel":   ps.rch.FlexModel(),
    "Berendrecht": ps.rch.Berendrecht(),
    "Direct":      None
}
response_functions = {
    "Exponential":       ps.Exponential(),
    "Gamma":             ps.Gamma(),
    "DoubleExponential": ps.DoubleExponential(),
    "Hantush":           ps.Hantush(),
    "FourParam":         ps.FourParam(),
}

# Containers for results
results = []
diagnostics_list = []

# Loop over each head‐series column and run recharge–response combinations
for head_name, head_ser in head_resampled.items():
    head_ser = head_ser.dropna()
    print(f"Processing head series: {head_name}")
    tmin, tmax = head_ser.index.min(), head_ser.index.max()

    for rch_name, rch_model in recharge_models.items():
        for rfunc_name, rfunc in response_functions.items():
            model_name = f"{head_name}_{rch_name}_{rfunc_name}"
            print(f"Running model: {model_name}")

            try:
                ml = ps.Model(head_ser, name=model_name)

                if rch_model is not None:
                    sm = ps.RechargeModel(
                        prec=prec_series,
                        evap=evap_series,
                        recharge=rch_model,
                        rfunc=rfunc,
                        name="rch"
                    )
                else:
                    sm = ps.StressModel(
                        recharge,
                        rfunc=rfunc,
                        name="direct"
                    )
                ml.add_stressmodel(sm)

                # Include autoregressive noise model
                ml.add_noisemodel(ps.ArNoiseModel())

                # Solve model over the head‐series timeframe
                ml.solve(tmin=tmin, tmax=tmax, solver=ps.LeastSquares(), report=False)

                # Collect statistics
                stats = ml.stats
                results.append({
                    "file":          head_name,
                    "model":         model_name,
                    "RechargeModel": rch_name,
                    "RechargeRfunc": rfunc_name,
                    "EVP":           stats.evp(),
                    "R2":            stats.rsq(),
                    "RMSE":          stats.rmse(),
                    "AIC":           stats.aic(),
                    "BIC":           stats.bic()
                })

                # Collect diagnostics
                diag = stats.diagnostics(alpha=0.05).copy()
                diag["model"] = model_name
                diagnostics_list.append(diag)

            except Exception as e:
                print(f"    Model {model_name} failed: {e}")
                results.append({
                    "file":          head_name,
                    "model":         model_name,
                    "RechargeModel": rch_name,
                    "RechargeRfunc": rfunc_name,
                    "EVP":           None,
                    "R2":            None,
                    "RMSE":          None,
                    "AIC":           None,
                    "BIC":           None,
                    "error":         str(e)
                })

# Assemble results into DataFrames
results_df = pd.DataFrame(results).sort_values("EVP", ascending=False)

if diagnostics_list:
    diagnostics_df = (
        pd.concat(diagnostics_list)
          .reset_index()
          .rename(columns={"index": "Test"})
          .loc[:, ["model","Test","Checks","Statistic","P-value","Reject H0 ($\\alpha$=0.05)"]]
    )
else:
    diagnostics_df = pd.DataFrame()

In [16]:
# 1. Load head‐series excels (skipping header row, parsing day‐first dates)
beemster_dir = os.path.join(project_root, "input_files", "input_beemster")

# Output directory
output_dir = os.path.join(project_root, "output_files") 
print(output_dir)

#  Save the results DataFrame as CSV and Excel
results_df.to_csv(os.path.join(output_dir, 'results_df.csv'), index=False)
results_df.to_excel(os.path.join(output_dir, 'results_df.xlsx'), index=False)

# Save the diagnostics DataFrame if it exists
diagnostics_df.to_csv(os.path.join(output_dir, 'diagnostics_df.csv'), index=False)
diagnostics_df.to_excel(os.path.join(output_dir, 'diagnostics_df.xlsx'), index=False)

c:\Users\jeroe\Desktop\Python\pastas-wv2030\output_files
